In [1]:
# ============================================================
# IMPORTS
# ============================================================
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import numpy as np
# ============================================================
# STEP 1: Load Food Access data
# ============================================================
df_food = pd.read_csv("FoodAccessResearchAtlasData2019.csv")
# Ensure CensusTract is string and zero-padded
df_food["CensusTract"] = df_food["CensusTract"].astype(str).str.zfill(11)
# Keep only needed columns
df_food = df_food[["CensusTract", "MedianFamilyIncome", "PovertyRate", "LAPOP1_10"]]

# ============================================================
# STEP 2: Load Massachusetts census tracts shapefile
# ============================================================
fips = "25"  # Massachusetts
tracts = gpd.read_file(f"https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_{fips}_tract.zip")
# Fix: Explicitly set and then re-project the CRS for tracts
tracts = tracts.set_crs("EPSG:4269", allow_override=True).to_crs("EPSG:4326")

# ============================================================
# STEP 3: Load restaurant data
# ============================================================
restaurants = pd.read_csv("filtered_restaurants_and_food(final).csv")
restaurants.columns = restaurants.columns.str.strip()
# Detect county column automatically
county_col = next((c for c in restaurants.columns if "county" in c.lower()), None)
# Debug: Check coordinate columns
print("Restaurant columns:", restaurants.columns.tolist())
print("\nFirst few rows of coordinates:")
print(restaurants[["Longitude", "Latitude"]].head())

# ============================================================
# STEP 4: Convert restaurants to GeoDataFrame and assign tracts
# ============================================================
# Make sure coordinates are numeric and not null
restaurants = restaurants.dropna(subset=["Longitude", "Latitude"])
restaurants["Longitude"] = pd.to_numeric(restaurants["Longitude"], errors='coerce')
restaurants["Latitude"] = pd.to_numeric(restaurants["Latitude"], errors='coerce')
restaurants = restaurants.dropna(subset=["Longitude", "Latitude"])
print(f"\n:white_check_mark: Loaded {len(restaurants)} restaurants with valid coordinates")
# Create GeoDataFrame - Longitude is X, Latitude is Y
restaurants_gdf = gpd.GeoDataFrame(
    restaurants,
    geometry=gpd.points_from_xy(restaurants["Longitude"], restaurants["Latitude"]),
    crs="EPSG:4326"
)
# Spatial join - this assigns each restaurant to its containing tract
restaurants_with_tract = gpd.sjoin(
    restaurants_gdf,
    tracts[["GEOID", "geometry"]],
    how="inner",  # Only keep restaurants that fall within a tract
    predicate="within"
)
restaurants_with_tract["CensusTract"] = restaurants_with_tract["GEOID"]
print(f":white_check_mark: Matched {len(restaurants_with_tract)} restaurants to census tracts")

# ============================================================
# STEP 5: Compute top 3 surplus restaurants per tract
# ============================================================
cols_needed = [
    "CensusTract",
    "UniqueID",
    "Name",
    "Address",
    "AvgExcessFoodWaste"
]
if county_col:
    cols_needed.insert(4, county_col)
# Filter to restaurants WITH a CensusTract assignment
restaurants_with_tract = restaurants_with_tract[cols_needed].dropna(subset=["CensusTract", "AvgExcessFoodWaste"])
# Sort by CensusTract and AvgExcessFoodWaste
restaurants_sorted = restaurants_with_tract.sort_values(
    ["CensusTract", "AvgExcessFoodWaste"],
    ascending=[True, False]
)
# Rank WITHIN each tract group
restaurants_sorted["Rank"] = restaurants_sorted.groupby("CensusTract")["AvgExcessFoodWaste"].rank(
    method="first",
    ascending=False
)
# Keep only top 3 per tract
top3 = restaurants_sorted[restaurants_sorted["Rank"] <= 3].copy()

# ============================================================
# STEP 5B: Pivot top 3 restaurants to wide format
# ============================================================
top3_pivot = top3.pivot_table(
    index="CensusTract",
    columns="Rank",
    values=["Name", "Address", "AvgExcessFoodWaste"],
    aggfunc="first"
)
# Flatten column names
top3_pivot.columns = [f"{col[0]}_Top{int(col[1])}" for col in top3_pivot.columns]
top3_pivot = top3_pivot.reset_index()
print(f":white_check_mark: Created top 3 restaurants for {len(top3_pivot)} census tracts")

# ============================================================
# STEP 6: Merge with food access data and tracts
# ============================================================
tracts_summary = tracts.merge(df_food, left_on="GEOID", right_on="CensusTract", how="left")
tracts_summary = tracts_summary.merge(top3_pivot, on="CensusTract", how="left")

# ============================================================
# STEP 7: Compute NeedScore (Improved Normalization)
# ============================================================
# ---------- Helper functions ----------
def scale_0_1(series):
    """Min–max scale to [0, 1] range, handling NaNs."""
    return (series - series.min()) / (series.max() - series.min())
def nonlinear_scale(series, alpha=2):
    """
    Apply exponential scaling to emphasize higher-need tracts.
    alpha > 0 increases contrast (e.g., alpha=2 or 3).
    """
    norm = scale_0_1(series.fillna(series.min()))
    return (np.exp(alpha * norm) - 1) / (np.exp(alpha) - 1)
# ---------- Normalize individual indicators ----------
# PovertyRate: higher → more need
tracts_summary["norm_poverty"] = nonlinear_scale(tracts_summary["PovertyRate"], alpha=2)
# LAPOP1_10: higher → more need (more low-access population)
tracts_summary["norm_lapop"] = nonlinear_scale(tracts_summary["LAPOP1_10"], alpha=2)
# MedianFamilyIncome: higher → less need (invert)
income_norm = scale_0_1(tracts_summary["MedianFamilyIncome"])
tracts_summary["norm_income_need"] = nonlinear_scale(1 - income_norm, alpha=2)
# ---------- Combine into final NeedScore ----------
tracts_summary["NeedScore"] = (
    0.5 * tracts_summary["norm_poverty"] +
    0.3 * tracts_summary["norm_lapop"] +
    0.2 * tracts_summary["norm_income_need"]
)
# Optional: Normalize NeedScore again to [0,1] for easier interpretation
tracts_summary["NeedScore"] = scale_0_1(tracts_summary["NeedScore"])
print(":white_check_mark: Computed improved NeedScore using nonlinear normalization")

# ============================================================
# STEP 8: Rename columns for output consistency
# ============================================================
rename_map = {
    "Name_Top1": "FoodProvider_1",
    "Name_Top2": "FoodProvider_2",
    "Name_Top3": "FoodProvider_3",
    "AvgExcessFoodWaste_Top1": "AvgSurplus_Provider1",
    "AvgExcessFoodWaste_Top2": "AvgSurplus_Provider2",
    "AvgExcessFoodWaste_Top3": "AvgSurplus_Provider3",
    "Address_Top1": "Address_Provider1",
    "Address_Top2": "Address_Provider2",
    "Address_Top3": "Address_Provider3"
}
tracts_summary = tracts_summary.rename(columns=rename_map)
# Define final columns (added PovertyRate)
columns_to_keep = [
    "CensusTract",
    "FoodProvider_1", "FoodProvider_2", "FoodProvider_3",
    "AvgSurplus_Provider1", "AvgSurplus_Provider2", "AvgSurplus_Provider3",
    "Address_Provider1", "Address_Provider2", "Address_Provider3",
    "MedianFamilyIncome",
    "PovertyRate",
    "NeedScore",
    "geometry"
]
# Keep only existing columns (avoid KeyErrors)
tracts_summary = tracts_summary[[c for c in columns_to_keep if c in tracts_summary.columns]]
# Drop rows where CensusTract is missing
tracts_summary = tracts_summary.dropna(subset=["CensusTract"])

# ============================================================
# STEP 9: Save GeoPackage for ArcGIS
# ============================================================
tracts_summary.to_file("tract_summary_top3_restaurants.gpkg", layer="tracts_summary", driver="GPKG")
print(":white_check_mark: Saved GeoPackage with geometries preserved")

# ============================================================
# STEP 10: Save CSV with centroid coordinates
# ============================================================
tracts_summary_csv = tracts_summary.copy()
tracts_summary_csv["Centroid_Lat"] = tracts_summary_csv.geometry.centroid.y
tracts_summary_csv["Centroid_Lon"] = tracts_summary_csv.geometry.centroid.x
tracts_summary_csv = tracts_summary_csv.drop(columns=["geometry"])
tracts_summary_csv.to_csv("tract_summary_top3_restaurants.csv", index=False)
print(":white_check_mark: Saved CSV with centroid coordinates")

Restaurant columns: ['Name', 'Naics Code Description', 'NAICS Code', 'Address', 'City', 'County', 'State', 'Zip Code', 'Excess Food Estimate, Low (tons per year)', 'Excess Food Estimate, High (tons per year)', 'UniqueID', 'Longitude', 'Latitude', 'AvgExcessFoodWaste']

First few rows of coordinates:
   Longitude   Latitude
0 -71.112600  42.369921
1 -71.014286  42.278453
2 -70.932417  42.079941
3 -71.449229  42.565285
4 -71.058747  42.359146

:white_check_mark: Loaded 7757 restaurants with valid coordinates
:white_check_mark: Matched 7757 restaurants to census tracts
:white_check_mark: Created top 3 restaurants for 1460 census tracts
:white_check_mark: Computed improved NeedScore using nonlinear normalization
:white_check_mark: Saved GeoPackage with geometries preserved
:white_check_mark: Saved CSV with centroid coordinates


/var/folders/3c/9y9b_5vd6z19v61y8f2qsgb40000gn/T/ipykernel_40585/4048820155.py:172: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tracts_summary_csv["Centroid_Lat"] = tracts_summary_csv.geometry.centroid.y
/var/folders/3c/9y9b_5vd6z19v61y8f2qsgb40000gn/T/ipykernel_40585/4048820155.py:173: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tracts_summary_csv["Centroid_Lon"] = tracts_summary_csv.geometry.centroid.x
